In [1]:
from keras.models import Sequential
from keras.layers import Dense

import numpy as np
# fix random seed for reproducibility
np.random.seed(42)

import pandas as pd

Using TensorFlow backend.


In [16]:
#Folder for the dataset
datasetFolder = '~/dbpedia2016/dataset/'

#Number of files
numberOfFiles = 256

#Test split
testSplit=0.25

In [18]:
def load_data(datasetFolder, datasetXFile, datasetYFile):
    print('Loading {} & {}'.format(datasetXFile, datasetYFile))
    # load file
    dataX=pd.read_csv(datasetFolder + datasetXFile, engine='c', na_filter=False)
    dataY=pd.read_csv(datasetFolder + datasetYFile, engine='c', na_filter=False)

    # delete the id fields
    del dataX['id']
    del dataY['id']

    return dataX, dataY

In [19]:
dataX, dataY = load_data(datasetFolder,'datasetX_1.csv', 'datasetY_1.csv')

Loading datasetX_1.csv & datasetY_1.csv


In [5]:
dataX.head()

,walk_1,walk_2,walk_3,walk_4,walk_5,walk_6,walk_7,walk_8,walk_9,walk_10,...,walk_3542,walk_3543,walk_3544,walk_3545,walk_3546,walk_3547,walk_3548,walk_3549,walk_3550,walk_3551
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
dataY.head()

,class_1,class_2,class_3,class_4,class_5,class_6,class_7,class_8,class_9,class_10,...,class_530,class_531,class_532,class_533,class_534,class_535,class_536,class_537,class_538,class_539
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
print("Input Features: {}".format(dataX.shape[1]))
print("Output Classes: {}".format(dataY.shape[1]))

Input Features: 3551
Output Classes: 539


In [20]:
# create model
logisticRegression = Sequential(name='Simple Logistic Regression')
logisticRegression.add(Dense(dataY.shape[1], input_dim=dataX.shape[1], activation='sigmoid'))

simpleModel = Sequential(name='2 Fully Connected Layers')
simpleModel.add(Dense(1024, input_dim=dataX.shape[1], activation='relu'))
simpleModel.add(Dense(dataY.shape[1], activation='sigmoid'))

deepModel = Sequential(name='Deep Model (5 Dense Layers)')
deepModel.add(Dense(2048, input_dim=dataX.shape[1], activation='relu'))
deepModel.add(Dense(1024, activation='relu'))
deepModel.add(Dense(768, activation='relu'))
deepModel.add(Dense(512, activation='relu'))
deepModel.add(Dense(dataY.shape[1], activation='sigmoid'))

models = [logisticRegression, simpleModel, deepModel]

In [50]:
# Compile model
import keras.backend as K

def count_predictions(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives, predicted_positives, possible_positives

def f1score(y_true, y_pred):
    true_positives, predicted_positives, possible_positives = count_predictions(y_true, y_pred)
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1score = 2.0 * precision * recall / (precision+recall+ K.epsilon())
    return f1score

for model in models:
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[f1score])

In [54]:
def fit_data(model, dataX, dataY):
    # Fit the model
    #model.fit(dataX.as_matrix(), dataY.as_matrix(), nb_epoch=5, verbose=2, batch_size=256)
    return model.train_on_batch(dataX.as_matrix(), dataY.as_matrix())

In [55]:
def countPredictions(y_true, y_pred):
    true_positives = np.sum(np.round(y_pred*y_true))
    predicted_positives = np.sum(np.round(y_pred))
    possible_positives = np.sum(y_true)
    return true_positives, predicted_positives, possible_positives

In [57]:
#Randomize the list of numbers so we can split train and test dataset
listOfFiles=list(range(1,numberOfFiles+1))
import random
random.shuffle(listOfFiles)
splitIndex=int((1-testSplit)*numberOfFiles)

numberOfEons = 15
for eon in range(0, numberOfEons):
    print('\n***********************************\nEon {}/{}'.format(eon+1, numberOfEons))
    for trainIndex in range(0,splitIndex):
        print('==========================================')
        print('Learning for file {} / {} : datasetX/Y_{}'.format(trainIndex+1, splitIndex, listOfFiles[trainIndex]))
        dataX, dataY = load_data(datasetFolder,'datasetX_{}.csv'.format(listOfFiles[trainIndex]), 'datasetY_{}.csv'.format(listOfFiles[trainIndex]))
        print('Training...')
        for model in models:
            #print('Model = {}'.format(model.name))
            loss, f1score=fit_data(model,dataX, dataY)
            print('   Model = {} \t loss = {:.4f} \t f1-score = {:.4f}'.format(model.name, loss, f1score))
            
        
    counts = {} 
    for model in models:
        counts[model.name] = {'true_positives':0, 'predicted_positives':0, 'possible_positives':0}
    
    print('==========================================\n')
    for testIndex in range(splitIndex, numberOfFiles):
        dataX, dataY = load_data(datasetFolder,'datasetX_{}.csv'.format(listOfFiles[testIndex]), 'datasetY_{}.csv'.format(listOfFiles[testIndex]))
        print ('Testing for file {} / {} : datasetX/Y_{}'.format(testIndex+1, numberOfFiles, listOfFiles[trainIndex]))
        for model in models:
            predY=model.predict_on_batch(dataX.as_matrix())
            true_positives, predicted_positives, possible_positives = countPredictions(dataY.as_matrix(), predY)
            counts[model.name]['true_positives'] += true_positives
            counts[model.name]['predicted_positives'] += predicted_positives
            counts[model.name]['possible_positives'] += possible_positives
    print('==========================================\n')
    
    
    print('{}. Eon {}/{} - Testing score:'.format(eon+1, eon+1, numberOfEons))
    for model in models:
        count = counts[model.name]
        precision = (count['true_positives'])/(count['predicted_positives']+0.0001)
        recall = (count['true_positives'])/(count['possible_positives']+0.0001)
        f1score = 2.0 * precision * recall / (precision+recall+0.0001)
        print(' - Model = {} \t f1-score = {:.4f}\t precision = {:.4f} \t recall = {:.4f}'.format(model.name, f1score, precision, recall))
    print('==========================================\n')


***********************************
Eon 1/5
Learning for file 1 / 192 : datasetX/Y_182
Loading datasetX_182.csv & datasetY_182.csv
Training...
   Model = Simple Logistic Regression 	 loss = 0.0895 	 f1-score = 0.5391
   Model = 2 Fully Connected Layers 	 loss = 0.0107 	 f1-score = 0.7067
   Model = Deep Model (5 Dense Layers) 	 loss = 0.0075 	 f1-score = 0.7867
Learning for file 2 / 192 : datasetX/Y_180
Loading datasetX_180.csv & datasetY_180.csv
Training...
   Model = Simple Logistic Regression 	 loss = 0.0914 	 f1-score = 0.5358
   Model = 2 Fully Connected Layers 	 loss = 0.0108 	 f1-score = 0.7014
   Model = Deep Model (5 Dense Layers) 	 loss = 0.0075 	 f1-score = 0.7860
Learning for file 3 / 192 : datasetX/Y_47
Loading datasetX_47.csv & datasetY_47.csv
Training...
   Model = Simple Logistic Regression 	 loss = 0.0908 	 f1-score = 0.5334
   Model = 2 Fully Connected Layers 	 loss = 0.0109 	 f1-score = 0.6987
   Model = Deep Model (5 Dense Layers) 	 loss = 0.0076 	 f1-score = 0.786

1. Eon 1/5 - Testing score: 
 - Model = Simple Logistic Regression 	 f1-score = 0.5630	 precision = 0.8697 	 recall = 0.4163
 - Model = 2 Fully Connected Layers 	 f1-score = 0.7933	 precision = 0.8852 	 recall = 0.7187
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8094	 precision = 0.8896 	 recall = 0.7425
2. Eon 2/5 - Testing score: 
 - Model = Simple Logistic Regression 	 f1-score = 0.5927	 precision = 0.8698 	 recall = 0.4495
 - Model = 2 Fully Connected Layers 	 f1-score = 0.8042	 precision = 0.8867 	 recall = 0.7358
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8120	 precision = 0.8884 	 recall = 0.7478
3. Eon 3/5 - Testing score: 
 - Model = Simple Logistic Regression 	 f1-score = 0.6137	 precision = 0.8675 	 recall = 0.4748
 - Model = 2 Fully Connected Layers 	 f1-score = 0.8083	 precision = 0.8874 	 recall = 0.7423
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8129	 precision = 0.8884 	 recall = 0.7493
4. Eon 4/5 -Testing score: 
 - Model = Simple Logistic Regression 	 f1-score = 0.6306	 precision = 0.8664 	 recall = 0.4958
 - Model = 2 Fully Connected Layers 	 f1-score = 0.8108	 precision = 0.8869 	 recall = 0.7468
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8135	 precision = 0.8881 	 recall = 0.7506
5. Eon 5/5 -Testing score: 
 - Model = Simple Logistic Regression 	 f1-score = 0.6454	 precision = 0.8665 	 recall = 0.5142
 - Model = 2 Fully Connected Layers 	 f1-score = 0.8120	 precision = 0.8870 	 recall = 0.7487
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.8151	 precision = 0.8864 	 recall = 0.7545